# Introduction

**Decorator:** a function that takes another function as an argument and returns another function.

## Decorators as Functions

In [6]:
def decorator_function(original_function):
    
    def wrapper_function():
        print('wrapper executed this before {}'.format(original_function.__name__))
        return original_function()
        
    return wrapper_function


def display():
    print('display function ran')

decorated_display = decorator_function(display)

decorated_display()

wrapper executed this before display
display function ran


## Why is this Useful?

Decorators allow us to wrap a function in order to extend the behavior of the wrapped function, without permanently modifying it.

Although the above syntax for creating a decorator is perfectly acceptable, the following is more common.

In order to make both the `display` and `display_info` functions work, we need to use `*args` and `**kwargs` to pass an indeterminant number of variables to the wrapper function. The `*args` argument handles non-keyworded arguments, while `**kwargs` handles keyworded arguments.

In [13]:
def decorator_function(original_function):
    
    def wrapper_function(*args, **kwargs):
        print('wrapper executed this before {}'.format(original_function.__name__))
        return original_function(*args, **kwargs)
        
    return wrapper_function

@decorator_function
def display():
    print('display function ran')
    
@decorator_function
def display_info(name, age):
    print('display_info ran with arguments ({}, {})'.format(name, age))
    

display()
display_info('John', 45)



wrapper executed this before display
display function ran
wrapper executed this before display_info
display_info ran with arguments (John, 45)


## Classes as Decorators

We can also use classes as decorators. In this case, we use the `__call__` method as the decorator.


In [4]:
class decorator_class(object):
    
    def __init__(self, original_function):
        self.original_function = original_function
        
    def __call__(self, *args, **kwargs):
        print('call method executed this before {}'.format(self.original_function.__name__))
        self.original_function(*args, **kwargs)
        print('call method executed this after {} \n'.format(self.original_function.__name__))
        return None
        
@decorator_class
def display():
    print('display function ran')
    
    
@decorator_class
def display_info(name, age):
    print('display_info ran with arguments ({}, {})'.format(name, age))
    

display()
display_info('John', 45)
        
        

call method executed this before display
display function ran
call method executed this after display 

call method executed this before display_info
display_info ran with arguments (John, 45)
call method executed this after display_info 



# Practical Decorator Example: Logging

Using decorators with loggers makes sense because we often want to log info throughout our codebase. By using a decorator, we are able to log the information in a consistent way with very little overhead.

People often use decorators with timers for the same reason. If we're interested in how long a function took to run, we can use a decorator for each function we want to keep track of. This allows us to time functions of interest with one line of code at instantiation without cluttering things up at execution.

When we stack our decorators on top of one another, what that is actually doing is chaining our decorators instead of applying each decorator to the function of interest. To avoid unexpected results, we need to wrap each of our decorated functions within the decorators using the `wraps` tool from the `functools` module.

In [6]:
from functools import wraps

def my_logger(orig_func):
    import logging
    logging.basicConfig(filename='../data/{}.log'.format(orig_func.__name__), level=logging.INFO)
    
    @wraps(orig_func)
    def wrapper(*args, **kwargs):
        logging.info(
            'Ran with args: {}, and kwargs: {}'.format(args, kwargs)
        )
        return orig_func(*args, **kwargs)
    
    return wrapper


def my_timer(orig_func):
    import time
    
    @wraps(orig_func)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = orig_func(*args, **kwargs)
        t2 = time.time() - t1
        print('{} ran in: {} sec'.format(orig_func.__name__, t2))
        return result
    
    return wrapper

@my_timer
@my_logger
def display_info(name, age):
    print('display_info ran with arguments ({}, {})'.format(name, age))
    
print(display_info.__name__)    
display_info('Hank', 30)
        

display_info
display_info ran with arguments (Hank, 30)
display_info ran in: 0.00044417381286621094 sec


In [7]:
with open('../data/display_info.log', 'rb') as f:
    for line in f:
        print(line)

b"INFO:root:Ran with args: ('Hank', 30), and kwargs: {}\n"
b"INFO:root:Ran with args: ('Hank', 30), and kwargs: {}\n"
b"INFO:root:Ran with args: ('Hank', 30), and kwargs: {}\n"
b"INFO:root:Ran with args: ('Hank', 30), and kwargs: {}\n"
b"INFO:root:Ran with args: ('Hank', 30), and kwargs: {}\n"
